In [4]:
%%capture output
%pip install numpy
%pip install matplotlib
%pip install torch

![images](images/GreenMaskAlg1.png)
![images](images/GreenMaskAlg2.png)

In [167]:
import torch
from torch.distributions.normal import Normal


class SMC():
    def __init__(self, n_threads: int, n_iterations: int):
        self.n_threads = n_threads
        self.n_iterations = n_iterations
        self.theta = [self.initial_theta(loc=0, scale=1)]
        self.weights = [self.initial_weights()]
        self.estimate = []
        print(f'smc initialised, use run_smc to run all iterations')
        return
    
    def target_dist_prob(self, th):
        normal = Normal(torch.tensor([5]),torch.tensor([0.5]))
        return torch.exp(normal.log_prob(th))

    def initial_weights(self):
        normal = Normal(torch.tensor([0]),torch.tensor([1]))
        # maybe this is stupid but idk why theres no way of getting prob
        # only thing that i can get returned is log prob instead
        prob_q = (torch.exp(normal.log_prob(self.theta[-1])))

        prob_target = self.target_dist_prob(self.theta[-1])
   
        return prob_target/prob_q
    
    def initial_theta(self, loc, scale):
        # maybe this is stupid but idk why theres no
        return torch.normal(loc, scale,size=(1,self.n_threads))[-1]
    
    def normalise_weights_(self):
        self.weights[-1] = self.weights[-1] / self.weights[-1].sum()
    
    def calc_neff(self):
        return 1 / (self.weights[-1] @ self.weights[-1].T)
    
    def new_weights_(self):
        target_new = self.target_dist_prob(self.theta[-1])
        target_old = self.target_dist_prob(self.theta[-2])
        self.weights.append(self.weights[-1] * (target_new/target_old))
        return
    
    def weighted_resample_(self):
        r"""
        in place chages theta and weights using a weighted resample
        """
        new_sample_indexes = torch.multinomial(self.weights[-1],self.n_threads,replacement=True)

        theta_hat = torch.tensor([self.theta[-1][i] for i in new_sample_indexes])
 
        self.theta[-1] = theta_hat
        self.weights[-1] = (torch.ones(1,self.n_threads)[-1])*(1/self.n_threads)

    def estimator_(self):
        self.estimate.append(self.theta[-1] @ self.weights[-1].T)

    def sample_(self):
        self.theta.append(self.theta[-1]+torch.normal(0,1,size=(1,self.n_threads))[-1])

    def run_smc(self):
        for _ in range(self.n_iterations):
            self.normalise_weights_()
            # estimate quals of interest
            self.estimator_()

            # assess if we need to resample
            neff = self.calc_neff()
            if neff<(self.n_threads/2):
                self.weighted_resample_()
            self.sample_()
            self.new_weights_()
        return

def main():
    n = 100
    t = 500
    smc_torch = SMC(n,t)
    smc_torch.run_smc()
    print(smc_torch.estimate)
    return

main()

smc initialised, use run_smc to run all iterations
[tensor(3.0225), tensor(4.5730), tensor(4.8642), tensor(4.9239), tensor(4.9408), tensor(4.9520), tensor(4.9452), tensor(4.8282), tensor(4.9852), tensor(5.0388), tensor(4.9631), tensor(4.8690), tensor(4.8601), tensor(4.7689), tensor(5.0031), tensor(4.8176), tensor(4.9664), tensor(4.9153), tensor(4.9050), tensor(4.8102), tensor(4.7860), tensor(4.7783), tensor(4.8916), tensor(5.0045), tensor(5.0449), tensor(4.9687), tensor(4.8562), tensor(5.0189), tensor(5.1008), tensor(5.1233), tensor(4.9787), tensor(4.9676), tensor(4.9212), tensor(5.1613), tensor(5.0879), tensor(4.9382), tensor(4.8678), tensor(4.7878), tensor(4.8079), tensor(5.0211), tensor(5.1208), tensor(4.9649), tensor(5.0070), tensor(5.0951), tensor(5.0293), tensor(5.1089), tensor(5.0958), tensor(5.0294), tensor(5.0230), tensor(5.0851), tensor(4.9760), tensor(4.9911), tensor(4.9758), tensor(5.0324), tensor(5.0074), tensor(5.0839), tensor(4.8563), tensor(4.9678), tensor(4.8290), tens

for large n its about 2-3 times slower than the numpy implementation, although idk how much the class stuff also slows it down, should make a general high dimensional case for both and run on my pc at home so as it acctaully has a gpu so should take advantage of cuda optimistaion